## This can be run on Google Colab notebooks

In [ ]:
#Import Libraries
from google.colab import files
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os
import zipfile
import tensorflow as tf
from google.colab import drive

In [ ]:
# Upload a zip file containing your train and test folders
uploaded = files.upload()

# Specify the name of the uploaded zip file
zip_file_name = next(iter(uploaded))

# Create a directory to extract the contents
extracted_dir = 'extracted_images'
os.makedirs(extracted_dir, exist_ok=True)

In [ ]:
# Mount Google Drive in the current Colab session
drive.mount('/content/drive')

In [ ]:
# Unzip the file
with zipfile.ZipFile(zip_file_name, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Define paths to train and test folders
train_folder = os.path.join(extracted_dir, 'New folder', 'Train')
test_folder = os.path.join(extracted_dir, 'New folder', 'Test')

# List the contents of the train and test folders
print(f'Contents of {train_folder}: {os.listdir(train_folder)}')
print(f'Contents of {test_folder}: {os.listdir(test_folder)}')

# Create data generators
batch_size = 64

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,         # Rescale pixel values to the range [0,1]
    shear_range=0.3,        # Shear augmentation with a shear intensity of 0.3
    zoom_range=0.3,         # Zoom augmentation with a zoom range of 0.3
    horizontal_flip=True    # Horizontal flip augmentation
)

# ImageDataGenerator for rescaling testing images
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches from the specified directory
train_generator = train_datagen.flow_from_directory(
    train_folder,                        # Path to the directory containing training images
    target_size=(224, 224),              # Resize images to (224, 224) pixels
    batch_size=batch_size,               # Number of images in each batch
    class_mode='binary'                  # Binary classification, assuming two classes
)

# Flow testing images in batches from the specified directory for validation
validation_generator = test_datagen.flow_from_directory(
    test_folder,            
    target_size=(224, 224), 
    batch_size=batch_size,  
    class_mode='binary'     
)

In [ ]:
# Load pre-trained ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = True

# Create your custom model
model = models.Sequential()
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.3))
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
epochs = 25
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)

In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test Loss: {test_loss:.4f}')
print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
# Save the entire model to a file
model.save('your_model.h5')

# Download the saved model file
files.download('your_model.h5')